## An Investigation to Reproduce the findings in:

## “Low-Resource” Text Classification: A Parameter-Free Classification Method with Compressors

#### Zhiying Jiang1,2, Matthew Y.R. Yang1, Mikhail Tsirlin1, Raphael Tang1, Yiqin Dai2 and Jimmy Lin1

#### Reproduction by Ian Snyder

The intuition of using compressors for classification is that:
- compressors are good at capturing regularity
- objects from the same category share more regularity than those from different categories.

In [2]:
import gzip
import numpy as np
from tqdm import tqdm

In [3]:
import pandas as pd 
data = pd.read_csv("/Users/iansnyder/Desktop/Projects/NER_proj/data/bbc-text.csv")

In [4]:
n = int(0.8 * len(data))
v = int(0.1 * len(data))
train_data = data[:n]
val_data = data[n:n+v]
labels = {"business": 0, "entertainment": 1, "sport": 2, "tech": 3, "politics": 4}
train_data = train_data.replace({"category": labels})
val_data = val_data.replace({"category": labels})
train_data = np.array([(row["text"], row["category"]) for _, row in train_data.iterrows()])
val_data = np.array([(row["text"], row["category"]) for _, row in val_data.iterrows()])

In [5]:
k=5
results = []
#for each (text,label) in validation set
for (x1, _) in tqdm(val_data):
    #calculate the compressed length of the utf-8 encoded text
    Cx1 = len(gzip.compress(x1.encode()))
    #create a distance array
    distance_from_x1 = []
    #for each (text,label) in training set
    for (x2,_) in train_data:
        #calculate the compressed length of the utf-8 encoded text
        Cx2 = len(gzip.compress(x2.encode()))
        #concatenate the two texts
        x1x2 = " ".join([x1, x2])
        #calculate the compressed length of the utf-8 encoded concatenated text
        Cx1x2 = len(gzip.compress(x1x2.encode()))
        #calculate the normalized compression distance: a normalized version of information distance
        ncd = (Cx1x2 - min(Cx1,Cx2)) / max(Cx1, Cx2)
        #append the distance to the distance array
        distance_from_x1.append(ncd)
    #sort the distance array and get the top k classes
    sorted_idx = np.argsort(np.array(distance_from_x1))
    #get the top k classes
    top_k_class = train_data[sorted_idx[:k], 1]
    #get the most frequent class based on the top k classes
    predict_class = max(set(top_k_class), key = list(top_k_class).count)
    #append the predicted class to the results array
    results.append(predict_class)
    


100%|██████████| 222/222 [02:40<00:00,  1.38it/s]


This intuition can be formalized as a distance met- ric derived from Kolmogorov complexity (Kol- mogorov, 1963). Kolmogorov complexity K (x) characterizes the length of the shortest binary pro- gram that can generate x. K(x) is theoretically the ultimate lower bound for information measurement.

The intuition behind using compressed length is that the length of x that has been maximally com- pressed by a compressor is close to K(x). Gener- ally, the higher the compression ratio, the closer C(x) is to K(x).

 C(x) here means the length of x af- ter being compressed by gzip. C(xy) is the com- pressed length of concatenation of x and y. With the distance matrix NCD provides, we can then use k-nearest-neighbor to perform classification.

In [6]:
#calculate the accuracy
accuracy = np.sum(results == val_data[:,1]) / len(val_data)
#calculate standard deviation
std = np.std(results == val_data[:,1])
print(f"The Accuracy is: {accuracy}\nThe Standard Deviation is: {std}")

# calculate the f1 score
from sklearn.metrics import f1_score
f1 = f1_score(val_data[:,1], results, average='macro')
print(f"The F1 Score is: {f1}")

The Accuracy is: 0.963963963963964
The Standard Deviation is: 0.18637982761781263
The F1 Score is: 0.9603174603174602


The Model has extremely impressive accuracy, outperforming the BERT model and only taking 2 minutes to run on a 2016 macbook pro with 2 GHz Dual-Core Intel Core i5. 

Compared to my BERT Fine-Tuning on the same dataset and same hardware, this method had equal accuracy, but ran 15 times faster and used 72,000 times less storage. Not to mention several hours saved with the minimal preprocessing. 